# Divide and Conquer

1. Divide into subproblems
2. Conquer using recursive calls
3. Combine solutions

## 1) Mergesort

In [1]:
def mergesort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = mergesort(arr[:mid])
    right = mergesort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    merged = []
    left_index, right_index = 0, 0
    
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged += left[left_index:]
    merged += right[right_index:]
    return merged

mergesort([1, 3, 6, 8, 9, 12, 4])

[1, 3, 4, 6, 8, 9, 12]

## 2) Count Inversions

```
Input: [1, 3, 5, 2, 4, 6]
Output: 3 => (3, 2) (5, 2), (5, 4)
```

In [2]:
def count_inversions(arr):
    if len(arr) <= 1:
        return arr, 0
    mid = len(arr) // 2
    left, left_inversions = count_inversions(arr[:mid])
    right, right_inversions = count_inversions(arr[mid:])
    merged, split_inversions = merge_and_count(left, right)
    return merged, left_inversions + right_inversions + split_inversions

def merge_and_count(left, right):
    merged = []
    split_inversions = 0
    left_index, right_index = 0, 0
    
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
            split_inversions += len(left) - left_index

    merged += left[left_index:]
    merged += right[right_index:]
    return merged, split_inversions

count_inversions([1, 3, 5, 2, 4, 6])

([1, 2, 3, 4, 5, 6], 3)

## 3) Strassen's Subcubic Matrix Multiplication

Compute matrix multiplication (here $n \times n$) using only seven products:

$$
X = 
\begin{bmatrix}
    A & B \\
    C & D
\end{bmatrix} \quad
Y = 
\begin{bmatrix}
    E & F \\
    G & H
\end{bmatrix}
$$
<br>

$$
P_1 = A(F - H) \quad
P_2 = (A + B)H \quad
P_3 = (C + D)E \quad
P_4 = D(G - E) \\
P_5 = (A + D)(E + H) \quad
P_6 = (B - D)(G + H) \quad
P_7 = (A - C)(E + F)
$$
<br>

$$
X Y =
\begin{bmatrix}
    AE + BG & AF + BH \\
    CE + DG & CF + DH
\end{bmatrix}
=
\begin{bmatrix}
    P_5 + P_4 - P_2 + P_6 & P_1 + P_2 \\
    P_3 + P_4 & P_1 + P_5 - P_3 - P_7
\end{bmatrix}
$$

----

## 4) Closest Pair

**Input**: Set $P$ of $n$ points in $\mathbb{R}^2$

**Output**: Pair of distinct points with smallest distance (euclidean).

Runs in $O(n\ log\ n)$

In [3]:
import math

def closest_pair(points):
    points_x = sorted(points, key=lambda point: point[0])
    points_y = sorted(points, key=lambda point: point[1])
    return _closest_pair(points_x, points_y)
    
def _closest_pair(points_x, points_y):
    if len(points_x) <= 3:
        return min_pair(points_x)
    mid = len(points_x) // 2
    # Left half of sorted points
    Q_x = points_x[:mid]
    Q_y = points_y[:mid]
    # Right half of sorted points
    R_x = points_x[mid:]
    R_y = points_y[mid:]
    # Closest points in left half
    p_1, q_1 = _closest_pair(Q_x, Q_y)
    # Closest points in right half
    p_2, q_2 = _closest_pair(R_x, R_y)
    # Closest points split on left and right half
    delta = min(distance(p_1, q_1), distance(p_2, q_2))
    p_3, q_3 = _closest_split_pair(points_x, points_y, delta)
    return min_pair([p_1, q_1, p_2, q_2, p_3, q_3])
    
def _closest_split_pair(points_x, points_y, delta):
    """ If p, q is closest pair and is split on left and right half
        and let S_y = area with [max_x = delta, max_x + delta], then:
        * d(p, q) < delta
        * p and q are members of S_y
        * p and q are at most 7 positions apart in S_y
    """
    # Largest x coord in left half of points
    x_max = points_x[len(points_x) // 2][0]
    S_y = [p for p in points_y if x_max - delta <= p[0] and p[0] <= x_max + delta]
    min_dist = delta
    best_pair = None, None
    for i in range(1, len(S_y) - 1):
        for j in range(1, min(7, len(S_y) - i)):
            p = S_y[i]
            q = S_y[i + j]
            if distance(p, q) < min_dist:
                best_pair = p, q
    return best_pair

def distance(p, q):
    if not p or not q:
        return math.inf
    return math.sqrt((p[0] - q[0]) ** 2 + (p[1] - q[1]) ** 2)

def min_pair(points):
    """ Bruteforce closest pair for max 3 points """
    min_dist = distance(points[0], points[1])
    best = points[0], points[1]
    for p_1 in points:
        for p_2 in points:
            if p_1 != p_2 and distance(p_1, p_2) < min_dist:
                best = p_1, p_2
    return best
    
# best: (7,6),(5,8)
points = [(0,0),(7,6),(2,20),(12,5),(16,16),(5,8),(19,7),(14,22),(8,19),(7,29),(10,11),(1,13)]
closest_pair(points)

((7, 6), (5, 8))

### Correctness of closestSplitPair

#### Claim:

Let $p \in Q$, $r \in R$ be split pair with $d(p, q) < \delta$

Then:

* **(A)** $p$, $q$ are members of $S_y$
* **(B)** $p$, $q$ are at most **7** positions apart in $S_y$

Conclusion:

* If closest pair of $P$ is split pair, *closestSplitPair* finds it.
* *ClosestPair* is correct and runs in $O(n\ log\ n)$ time.

#### Proof of (A)

Let $p = (x_1, y_1) \in Q$, $q = (x_2, y_2) \in R$, $d(p, q) < \delta$

* Since $d(p, q) < \delta \Rightarrow \vert x_1 - x_2 \vert < \delta \land \vert y_1 - y_2 \vert < \delta$
* $p \in Q \Rightarrow x_1 \leq \bar{x} \land q \in R \Rightarrow x_2 \geq \bar{x}$
* $\Rightarrow x_1, x_2 \in [\bar{x} - \delta, \bar{x} + \delta] = P_y$

#### Proof of (B)

Grid with $\frac{\delta}{2} \times \frac{\delta}{2}$ boxes (8 boxes), with center $\bar{x}$ and bottom $min(y_1, y_2)$

**Lemma 1:** All points of $S_y$ with y-coords between those of $p$ and $q$ lie in one of these 8 boxes.

**Proof 1:** Recall that:

1. y-coords of $p$ and $q$ differ by $< \delta$
2. By definition of $S_y$, all points have x-coords between $\bar{x} - \delta$ and $\bar{x} + \delta$

**Lemma 2:** At most one point of $P$ in each box.

**Proof 2:**

By contradiction: Suppose $a$, $b$ lie in same box.

Then:

1. $a$ and $b$ are either both in $Q$ or both in $R$
2. $d(a, b) \leq \frac{\delta}{2} * \sqrt{2} < \delta$

But:

1 and 2 contradict definition of $\delta$ (min distance between of points in $Q$ or $R$).

$\Rightarrow$ Positions of $p$, $q$ in $S_y$ differ by at most 7.

----

## 5) Master Theorem

"Black box" for solving recurrences.

**Assumption:** All subproblems have equal size.

### Recurrence Format

**1) Base Case:** $T(n) \leq \text{a constant for all sufficiently small n}$

**2) For all larger n:**

$ T(n) \leq a T(\frac{n}{b} + O(n^d))$

Where:

* $a$ = number of recursive calls ($\leq 1$)
* $b$ = input size shrinkage factor ($> 1$)
* $d$ = exponent in running time of "combine steps" ($\leq 0$)
* $\Rightarrow$ $a$, $b$, $d$ independent of $n$

### Recurrence Runtime

$T(n)=
  \left\{
    \begin{array}{lr}
      \text{Case 1:}\ O(n^d \log n) \iff a = b^d \\
      \text{Case 2:}\ O(n^d) \iff a < b^d \\
      \text{Case 3:}\ O(n^{log_b a}) \iff a > b^d
    \end{array}
  \right\}$
  
### Examples

#### Merge Sort

$a = 2$, $b = 2$, $d = 1$ $\Rightarrow$ Case 1: $T(n) = O(n \log n)$

#### Binary Search

$a = 1$, $b = 2$, $d = 0$ $\Rightarrow$ Case 1: $T(n) = O(\log\ n)$

#### Naive Integer Multiplication

$a = 4$, $b = 2$, $d = 1$ $\Rightarrow$ Case 3: $T(n) = O(n^{\log_2 4}) = O(n^2)$

### Intuition

$\text{Work at level j} \leq a^j c (\frac{n}{b^j})^d = c n^d (\frac{a}{b^d})^j$

$\text{Total work} \leq c n^d \sum_{j = 0}^{\log_b n} (\frac{a}{b^d})^j$

$a$ = Rate of subproblem proliferation (RSP)  
$b^d$ = Rate of work shrinkage per subproblem (RWS)

1) If **RSP = RWS**:

* The amount of work is the same at each level.
* Expect $O(n^d log n)$.

2) If **RSP < RWS**:

* The amount of work is decreasing with each level.
* Most work is done at top level. Root node dominates.
* Expect $O(n^d)$.

3) If **RSP > RWS**:

* The amount of work is increasing with each level.
* Most work is done at leaves. Runtime depends on number of leave nodes.
* Expect $O(n^{\log_b a})$.

### Proof

$
\text{Total work} \leq c n^d \sum_{j = 0}^{\log_b n} (\frac{a}{b^d})^j \quad (*)
$


#### Note on Sums

For $r \neq 1$: $1 + r + r^2 + ... + r^k = \frac{r^{k + 1} - 1}{r - 1}$

1) If $r < 1$ is constant, right hand side is $\leq \frac{1}{1 - r}$  
(i.e. first term of sum dominates)

2) If $r > 1$ is constant, right hand side $\leq r^k (1 + \frac{1}{r - 1})$  
(i.e. last term of sum dominates)

#### 1) If $a = b^d$, then:

$\quad(*) = c n ^d (\log_b n + 1) = O(n^d \log n)$

#### 2) If $a < b^d$, then:

$\quad r = \frac{a}{b^d} < 1$

$\quad (*) = O(n^d)$

#### 3) If $a > b^d$, then:

$\quad \frac{a}{b^d} = r > 1$

$\quad (*) \leq constant * largest term$

$\quad (*) = O(n^d * \frac{a}{b^d}^{\log_b n}) = O(a^{\log_b n})$

$\quad \text{since}\ b^{-d \log_b n} = (b^{\log_b n})^{-d} = n^{-d}$